In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
pth="dbfs:/FileStore/shared_uploads/ramakrishnaprasadsivangula@gmail.com/sharemarket-1.csv"
schm=StructType([
             StructField('MARKET', StringType(),True),
             StructField('SERIES', StringType(), True),
             StructField('SYMBOL', StringType(), True),
             StructField('SECURITY', StringType(), True),
             StructField('PREV_CL_PR', DoubleType(), True),
             StructField('OPEN_PRICE', DoubleType(), True),
             StructField('HIGH_PRICE', DoubleType(), True),
             StructField('LOW_PRICE', DoubleType(), True),
             StructField('CLOSE_PRICE', DoubleType(), True),
             StructField('T_TRDVAL', DoubleType(), True),
             StructField('NET_TRDQTY', IntegerType(), True),
             StructField('CORP_IND', StringType(), True),
             StructField('TRADES', IntegerType(), True),
             StructField('HI_52_WK', DoubleType(), True),
             StructField('LO_52_WK', DoubleType(), True)    
             ])
df = spark.read.csv(schema=schm,path=pth)
df.printSchema()

root
 |-- MARKET: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- SYMBOL: string (nullable = true)
 |-- SECURITY: string (nullable = true)
 |-- PREV_CL_PR: double (nullable = true)
 |-- OPEN_PRICE: double (nullable = true)
 |-- HIGH_PRICE: double (nullable = true)
 |-- LOW_PRICE: double (nullable = true)
 |-- CLOSE_PRICE: double (nullable = true)
 |-- T_TRDVAL: double (nullable = true)
 |-- NET_TRDQTY: integer (nullable = true)
 |-- CORP_IND: string (nullable = true)
 |-- TRADES: integer (nullable = true)
 |-- HI_52_WK: double (nullable = true)
 |-- LO_52_WK: double (nullable = true)



In [0]:
df.show(4)

+------+------+------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
|MARKET|SERIES|SYMBOL|            SECURITY|PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|  T_TRDVAL|NET_TRDQTY|CORP_IND|TRADES|HI_52_WK|LO_52_WK|
+------+------+------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
|     N|    N1|  IRFC|BOND 8.00% PA TAX...|    1086.0|    1085.0|    1085.0|  1084.54|     1085.0|3371959.44|      3108|        |     8|  1194.0|  1050.0|
|     N|    N1|  JNPT|BOND 6.82% PA TAX...|    1001.0|    1007.1|    1015.0|   1007.1|     1015.0|   51370.0|        51|        |     2| 1529.99|  1000.0|
|     N|    N1|  NHAI|BOND 8.20% PA TAX...|   1091.38|    1092.0|    1092.0|   1087.0|    1089.98|3861145.49|      3544|        |    39|  1140.0|  1045.0|
|     N|    N1|  NTPC|8.41%S-R-NCD SERI...|   1198.99|    1135.0|    1

In [0]:
#1.Query to display the number of series present in the data.
df.select("series").distinct().count()

Out[8]: 53

In [0]:
#2.Display the series present in the data.
df.select("series").show()

+------+
|series|
+------+
|    N1|
|    N1|
|    N1|
|    N1|
|    N1|
|    N2|
|    N2|
|    N2|
|    N2|
|    N2|
|    N2|
|    N2|
|    N2|
|    N2|
|    N2|
|    N3|
|    N3|
|    N3|
|    N4|
|    N4|
+------+
only showing top 20 rows



In [0]:
#3.Find the sum of all the prices in the each series.
df.groupBy('series').agg(round(sum("PREV_CL_PR"),2).alias("tot_PREV_CL_PR"),round(sum("OPEN_PRICE"),2).alias("tot_OPEN_PRICE"),round(sum("HIGH_PRICE"),2).alias("tot_HIGH_PRICE"),round(sum("LOW_PRICE"),2).alias("tot_LOW_PRICE"),round(sum("CLOSE_PRICE"),2).alias("tot_CLOSE_PRICE")).show()

+------+--------------+--------------+--------------+-------------+---------------+
|series|tot_PREV_CL_PR|tot_OPEN_PRICE|tot_HIGH_PRICE|tot_LOW_PRICE|tot_CLOSE_PRICE|
+------+--------------+--------------+--------------+-------------+---------------+
|    YH|        1021.0|        1021.0|       1028.01|       1021.0|        1028.01|
|    NS|        1025.6|        1025.0|        1040.0|       1025.0|         1039.0|
|    NL|       3395.97|       3430.58|        3431.5|       3368.1|         3380.9|
|    NK|       1940.51|        1927.0|       1976.75|      1912.01|        1915.72|
|    Z4|       1020.65|        1040.0|        1040.0|       1022.1|         1022.1|
|    NJ|       4296.79|        4285.7|       4296.99|       4281.0|        4291.49|
|    NX|         349.6|        349.35|        349.35|       282.11|         295.07|
|    YS|        1140.0|        1130.0|        1135.0|       1130.0|         1135.0|
|    NW|        1088.1|       1089.05|       1089.05|       1089.0|         

In [0]:
df.select(max("T_TRDVAL")).head()

Out[49]: Row(max(T_TRDVAL)=9604082.74)

In [0]:
#4.Display security,series with highest net trade value
hg=df.select(max("T_TRDVAL")).head()[0]
df.filter(col('T_TRDVAL')==hg).show()



+------+------+------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
|MARKET|SERIES|SYMBOL|            SECURITY|PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|  T_TRDVAL|NET_TRDQTY|CORP_IND|TRADES|HI_52_WK|LO_52_WK|
+------+------+------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
|     N|    N6|  NHAI|8.75% TAX FREE TR...|   1277.08|    1287.0|    1290.0|   1287.0|    1287.58|9604082.74|      7456|        |    67| 1406.97|  1247.0|
+------+------+------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+



In [0]:
#5.Display the series whose sum of all prices greater than the net trade value.
df.select(col("series"),col("T_TRDVAL"),(col("OPEN_PRICE")+col("HIGH_PRICE")+col("LOW_PRICE")+col("CLOSE_PRICE")).alias('sm')).filter(col('sm')>col('T_TRDVAL')).show()

+------+--------+-------+
|series|T_TRDVAL|     sm|
+------+--------+-------+
|    N2|  3666.0| 4888.0|
|    N4| 2044.02|4088.04|
|    N5|  1912.4| 3824.8|
|    N6|  1341.0| 5364.0|
|    NB|  1843.0| 3686.0|
+------+--------+-------+



In [0]:
#6.Display the series with highest net trade quantity.
hqty=df.select(max("NET_TRDQTY")).head()[0]
df.filter(col('NET_TRDQTY')==hqty).select(col('series'),col('NET_TRDQTY')).show()

+------+----------+
|series|NET_TRDQTY|
+------+----------+
|    N3|    143810|
+------+----------+



In [0]:
#7. Display the highest and lowest open price
df.select(max('open_price'),min('open_price')).show()

+---------------+---------------+
|max(open_price)|min(open_price)|
+---------------+---------------+
|        11450.0|          13.75|
+---------------+---------------+



In [0]:
#8.Query to display the series which have trades more than 80.
df.filter(col('trades')>80).select(col('series'),col('trades')).show()

+------+------+
|series|trades|
+------+------+
|    N2|   107|
|    N3|   787|
|    N7|   106|
+------+------+



In [0]:
#9.Display the difference between the net trade value net trade quantity for each series.
df.select(col('series'),col('T_TRDVAL'),col('NET_TRDQTY'),(col('T_TRDVAL')-col('NET_TRDQTY')).alias('diff')).show()

+------+----------+----------+----------+
|series|  T_TRDVAL|NET_TRDQTY|      diff|
+------+----------+----------+----------+
|    N1|3371959.44|      3108|3368851.44|
|    N1|   51370.0|        51|   51319.0|
|    N1|3861145.49|      3544|3857601.49|
|    N1|  113500.0|       100|  113400.0|
|    N1|  108530.0|       100|  108430.0|
|    N2| 677372.39|     20461| 656911.39|
|    N2|    3666.0|         3|    3663.0|
|    N2|    4548.0|         4|    4544.0|
|    N2|  423116.5|       388|  422728.5|
|    N2| 125060.45|       101| 124959.45|
|    N2|  464590.5|       370|  464220.5|
|    N2|  128275.0|       100|  128175.0|
|    N2| 526800.91|       526| 526274.91|
|    N2|  238400.0|       200|  238200.0|
|    N2| 1105942.5|      1048| 1104894.5|
|    N3|4233217.99|    143810|4089407.99|
|    N3|   78158.0|        78|   78080.0|
|    N3|1729096.92|      1639|1727457.92|
|    N4|   2044.02|         2|   2042.02|
|    N4|   29646.6|        30|   29616.6|
+------+----------+----------+----